# Import libraries and Config

In [ ]:
# import libraries
import osmnx as ox
import pandas as pd

In [ ]:
from Code_Jessica.functions import (
    add_edge_traffic_times,
    get_turn_penalty_dict,
    shortest_path_turn_penalty,
)

In [ ]:
def get_routes_from_OD_df(G, OD_df, **kwargs):
    routes = []
    for i in range(len(OD_df)):
        # find routes while considering penalties
        route = shortest_path_turn_penalty(G, OD_df.iloc[i]["oid"], OD_df.iloc[i]["did"], **kwargs)
        routes.append(route)
    return routes

In [ ]:
TRAFFIC_TIME_CONFIG = {
    "traffic_signals_time": 2,
    "stop_time": 2,
    "turning_circle_time": 0,
    "crossing_time": 1.5,
    "give_way_time": 1.5,
    "mini_roundabout_time": 1.5,
}

# Import OD pairs

In [ ]:
file_path = "../Data/"
output_file_path = file_path + "Output/"
OD_pairs = pd.read_csv(output_file_path + "temp_intersect_OD.csv")

In [ ]:
OD_pairs

# Import Street Network in LA Urban Area

In [ ]:
# using the street network of Graphml file of LA urban area
G = ox.io.load_graphml(file_path + "los_angeles-14.graphml")

In [ ]:
# add attributes
G = ox.add_edge_speeds(G)
G = ox.add_edge_travel_times(G)
G = ox.bearing.add_edge_bearings(G)

In [ ]:
G = add_edge_traffic_times(G, **TRAFFIC_TIME_CONFIG)

In [ ]:
nodes_gdf, edges_gdf = ox.graph_to_gdfs(G)

# Set Turn Penalty Parameter

In [ ]:
penalty = get_turn_penalty_dict(G, left_turn_penalty=30, right_turn_penalty=10, u_turn_penalty=90)

# Sample OD pair and calculate naive/free-flow VS penalized travel time

In [ ]:
# due to the run time, I only sampled 50 OD pairs
OD_pairs_sample = OD_pairs.sample(50, replace=True).copy()

In [ ]:
penalty_routes = get_routes_from_OD_df(G, OD_pairs_sample, weight="total_time", penalty=penalty)

In [ ]:
naive_routes = get_routes_from_OD_df(G, OD_pairs_sample, weight="travel_time")

In [ ]:
# travel time of 50 O-D pairs routes
penalty_time_result = []
naive_time_result = []

for i in range(len(penalty_routes)):
    x = sum(ox.utils_graph.get_route_edge_attributes(G, penalty_routes[i], "total_time"))
    y = sum(ox.utils_graph.get_route_edge_attributes(G, naive_routes[i], "travel_time"))
    penalty_time_result.append(x)
    naive_time_result.append(y)

In [ ]:
# compare the naive/freeflow travel time and penalized travel time
time_compare = pd.DataFrame(
    {
        "penalized travel time": penalty_time_result,
        "naive travel time": naive_time_result,
    },
)

In [ ]:
time_compare

In [ ]:
time_compare["penalized travel time"].sum() / time_compare["naive travel time"].sum()